<a href="https://colab.research.google.com/github/sanketghosh19/Crypto_Price_Prediction/blob/main/Stock_Clustering_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 1. Load the Excel file
from google.colab import files
uploaded = files.upload()  # For Colab environment
file_name = list(uploaded.keys())[0]
data = pd.ExcelFile(file_name)

# 2. Combine all sheets into one DataFrame.
#    The sheet name (e.g., "tcs.ns", "rvnl.ns") becomes the 'Stock' level in a MultiIndex.
all_data = {}
for sheet_name in data.sheet_names:
    try:
        sheet_data = data.parse(sheet_name)
        all_data[sheet_name] = sheet_data
    except Exception as e:
        print(f"Error reading sheet {sheet_name}: {e}")

combined_data = pd.concat(all_data.values(), keys=all_data.keys(), names=["Stock", "Index"])

# 3. Function to extract features from each stock's DataFrame
def extract_features(df):
    """
    Expects a DataFrame with a 'Close' column (OHLCV data).
    Computes various metrics (mean, std, min, max, volatility, total price change).
    """
    # Drop missing values in 'Close'
    close_prices = df['Close'].dropna().values
    if len(close_prices) < 2:
        return None

    mean_price = np.mean(close_prices)
    std_price = np.std(close_prices)
    min_price = np.min(close_prices)
    max_price = np.max(close_prices)

    # Volatility = (std / mean)
    # Price change = (last_price - first_price) / first_price
    features = {
        'mean_price': mean_price,
        'std_price': std_price,
        'min_price': min_price,
        'max_price': max_price,
        'volatility': (std_price / mean_price) if mean_price != 0 else 0,
        'price_change': ((close_prices[-1] - close_prices[0]) / close_prices[0]) if close_prices[0] != 0 else 0,
    }
    return features

# 4. Extract features for each stock (sheet).
feature_data = []
for stock_symbol, stock_df in combined_data.groupby(level=0):
    features = extract_features(stock_df)
    if features:
        features['Stock'] = stock_symbol
        feature_data.append(features)

# 5. Convert the collected feature dicts into a DataFrame.
features_df = pd.DataFrame(feature_data)

# 6. Normalize the numeric features (excluding 'Stock')
scaler = StandardScaler()
numeric_cols = ['mean_price', 'std_price', 'min_price', 'max_price', 'volatility', 'price_change']
features_normalized = scaler.fit_transform(features_df[numeric_cols])

# 7. Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
features_df['Cluster'] = kmeans.fit_predict(features_normalized)

# 8. Map clusters to probability groups based on average price_change
cluster_order_by_performance = (
    features_df.groupby('Cluster')['price_change']
    .mean()
    .sort_values(ascending=False)
    .index
)

# Example mapping (highest price_change => '90% Uptrend', etc.)
cluster_mapping = {
    cluster_order_by_performance[0]: '90% Uptrend',
    cluster_order_by_performance[1]: '80% Uptrend',
    cluster_order_by_performance[2]: '70% Uptrend'
}

features_df['Probability_Group'] = features_df['Cluster'].map(cluster_mapping)

# 9. Select exactly 20 stocks per probability group (if available)
def pick_top_20(df_group):
    if len(df_group) >= 20:
        return df_group.sample(n=20, random_state=42)
    else:
        return df_group.head(20)

final_clusters = (
    features_df.groupby('Probability_Group', group_keys=False)
    .apply(pick_top_20)
)

# 10. Display or save the final clusters
print(final_clusters)

# Save to CSV
final_clusters.to_csv("final_clustered_stocks.csv", index=False)

# Download the file (if using Colab)
files.download("final_clustered_stocks.csv")


Saving stock_data_daily (1).xlsx to stock_data_daily (1).xlsx


ValueError: n_samples=1 should be >= n_clusters=3.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 1. Load the Excel file
from google.colab import files
uploaded = files.upload()  # For Colab environment
file_name = list(uploaded.keys())[0]
data = pd.ExcelFile(file_name)

# 2. Combine all sheets into one DataFrame.
#    The sheet name (e.g., "tcs.ns", "rvnl.ns") becomes the 'Stock' level in a MultiIndex.
all_data = {}
for sheet_name in data.sheet_names:
    try:
        sheet_data = data.parse(sheet_name)
        all_data[sheet_name] = sheet_data
    except Exception as e:
        print(f"Error reading sheet {sheet_name}: {e}")

combined_data = pd.concat(all_data.values(), keys=all_data.keys(), names=["Stock", "Index"])

# 3. Function to extract features from each stock's DataFrame
def extract_features(df):
    """
    Expects a DataFrame with a 'Close' column (OHLCV data).
    Computes various metrics (mean, std, min, max, volatility, total price change).
    """
    # Drop missing values in 'Close'
    close_prices = df['Close'].dropna().values
    if len(close_prices) < 2:
        return None

    mean_price = np.mean(close_prices)
    std_price = np.std(close_prices)
    min_price = np.min(close_prices)
    max_price = np.max(close_prices)

    # Volatility = (std / mean)
    # Price change = (last_price - first_price) / first_price
    features = {
        'mean_price': mean_price,
        'std_price': std_price,
        'min_price': min_price,
        'max_price': max_price,
        'volatility': (std_price / mean_price) if mean_price != 0 else 0,
        'price_change': ((close_prices[-1] - close_prices[0]) / close_prices[0]) if close_prices[0] != 0 else 0,
    }
    return features

# 4. Extract features for each stock (sheet).
feature_data = []
for stock_symbol, stock_df in combined_data.groupby(level=0):
    features = extract_features(stock_df)
    if features:
        features['Stock'] = stock_symbol
        feature_data.append(features)

# 5. Convert the collected feature dicts into a DataFrame.
features_df = pd.DataFrame(feature_data)

# 6. Normalize the numeric features (excluding 'Stock')
numeric_cols = ['mean_price', 'std_price', 'min_price', 'max_price', 'volatility', 'price_change']
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features_df[numeric_cols])

# 7. Apply K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
features_df['Cluster'] = kmeans.fit_predict(features_normalized)

# 8. Map clusters to probability groups based on average price_change
cluster_order_by_performance = (
    features_df.groupby('Cluster')['price_change']
    .mean()
    .sort_values(ascending=False)
    .index
)

cluster_mapping = {
    cluster_order_by_performance[0]: '90% Uptrend',
    cluster_order_by_performance[1]: '80% Uptrend',
    cluster_order_by_performance[2]: '70% Uptrend'
}

features_df['Probability_Group'] = features_df['Cluster'].map(cluster_mapping)

# 9. Define a function to pick 20 items per group
def pick_top_20(df_group):
    if len(df_group) >= 20:
        return df_group.sample(n=20, random_state=42)
    else:
        return df_group.head(20)

# 10. Group by 'Probability_Group' and select top 20 per group
#     We add 'include_groups=True' so that grouping columns stay in the data,
#     addressing the DeprecationWarning.
final_clusters = (
    features_df.groupby('Probability_Group', group_keys=False)
    .apply(pick_top_20)
    .reset_index(drop=True)  # Reset index to keep 'Probability_Group' as a column
)
# Display or save the final clusters
print(final_clusters)

# Save to CSV
final_clusters.to_csv("final_clustered_stocks.csv", index=False)

# Download the file (if using Colab)
files.download("final_clustered_stocks.csv")


Saving stock_data_daily (1).xlsx to stock_data_daily (1) (1).xlsx


ValueError: n_samples=1 should be >= n_clusters=3.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from google.colab import drive, files

# 1. Mount Google Drive and specify the path to your Excel file
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/stock_data_daily.xlsx'  # Update this path as needed
data = pd.ExcelFile(file_path)

# 2. Combine all sheets into one DataFrame.
#    The sheet name (e.g., "tcs.ns", "rvnl.ns") becomes the 'Stock' level in a MultiIndex.
all_data = {}
for sheet_name in data.sheet_names:
    try:
        sheet_data = data.parse(sheet_name)
        all_data[sheet_name] = sheet_data
    except Exception as e:
        print(f"Error reading sheet {sheet_name}: {e}")

combined_data = pd.concat(all_data.values(), keys=all_data.keys(), names=["Stock", "Index"])

# 3. Function to extract features for each stock
def extract_features(df):
    """
    Expects a DataFrame with:
      - 'Close' column (daily close prices)
      - 'Volume' column (daily volumes)
    STRIKE PRICE (demonstration): We'll assume a constant strike price defined as the first day's close.
    """
    # Drop missing values in 'Close' and 'Volume'
    close_prices = df['Close'].dropna().values
    volume_data = df['Volume'].dropna().values

    # Ensure we have enough data
    if len(close_prices) < 2:
        return None

    # Strike Price Adjustment: Using the first day's close as the strike price.
    strike_price = close_prices[0]
    strike_diff_mean = np.mean(close_prices - strike_price)

    # Volume Feature: Average volume over the time span.
    avg_volume = np.mean(volume_data)

    # Price-Based Features
    mean_price = np.mean(close_prices)
    std_price = np.std(close_prices)
    min_price = np.min(close_prices)
    max_price = np.max(close_prices)
    price_change = ((close_prices[-1] - close_prices[0]) / close_prices[0]) if close_prices[0] != 0 else 0

    features = {
        'mean_price': mean_price,
        'std_price': std_price,
        'min_price': min_price,
        'max_price': max_price,
        'price_change': price_change,
        'strike_diff_mean': strike_diff_mean,
        'avg_volume': avg_volume
    }
    return features

# 4. Build a features DataFrame, one row per stock
feature_data = []
for stock_symbol, stock_df in combined_data.groupby(level=0):
    feats = extract_features(stock_df)
    if feats:
        feats['Stock'] = stock_symbol
        feature_data.append(feats)

features_df = pd.DataFrame(feature_data)

# 5. Choose the final features (including strike price adjustment and volume)
final_feature_cols = [
    'mean_price',
    'std_price',
    'min_price',
    'max_price',
    'price_change',
    'strike_diff_mean',
    'avg_volume'
]

# 6. Normalize the selected features
scaler = StandardScaler()
X = scaler.fit_transform(features_df[final_feature_cols])

# 7. Clustering
if X.shape[0] < 2:
    print("Only one sample available for clustering. Skipping clustering step.")
    features_df['Cluster'] = 0
    best_k = 1
    best_score = 0
    best_kmeans = None
else:
    best_k = None
    best_score = -1
    best_kmeans = None
    for k in range(2, 11):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X)
        score = silhouette_score(X, labels)
        if score > best_score:
            best_score = score
            best_k = k
            best_kmeans = kmeans

    if best_score < 0.95:
        print(f"WARNING: Highest silhouette score is {best_score:.4f} with k={best_k}. Could not reach 95% threshold.")
    else:
        print(f"Selected k={best_k} with a silhouette score of {best_score:.4f} (>= 0.95).")
    features_df['Cluster'] = best_kmeans.labels_

# 8. Map clusters to uptrend groups (e.g., 95%, 90%, 80% Uptrend)
#    Sort clusters by average price_change in descending order
avg_change = features_df.groupby('Cluster')['price_change'].mean().sort_values(ascending=False)
cluster_order_by_performance = avg_change.index

uptrend_labels = ['95% Uptrend', '90% Uptrend', '80% Uptrend']
cluster_mapping = {}
for i, cluster_id in enumerate(cluster_order_by_performance):
    if i < len(uptrend_labels):
        cluster_mapping[cluster_id] = uptrend_labels[i]
    else:
        cluster_mapping[cluster_id] = '80% Uptrend (Extra)'

features_df['Probability_Group'] = features_df['Cluster'].map(cluster_mapping)

# 9. Select exactly 20 stocks per group (or as many as available)
def pick_top_20(df_group):
    if len(df_group) >= 20:
        return df_group.sample(n=20, random_state=42)
    else:
        return df_group.head(20)

final_clusters = (
    features_df.groupby('Probability_Group', group_keys=False)
    .apply(pick_top_20)
    .reset_index(drop=True)
)

# 10. Display and save results
print(final_clusters)

output_filename = "final_clustered_stocks.csv"
final_clusters.to_csv(output_filename, index=False)
files.download(output_filename)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     mean_price    std_price   min_price     max_price  price_change  \
0     26.056024    36.866617    0.917223    179.940002     71.639321   
1    132.350672   120.663265    6.544371    514.626953     43.407183   
2    220.378835   265.073614    7.759712   1039.099976    125.538209   
3    270.014436   241.107171    9.826268    817.318909     37.947364   
4    361.267470   411.626491    9.246290   1595.484985    106.231976   
5    519.336229   419.255882   39.605797   1796.400024     15.069284   
6    347.039372   326.387437    3.332827   1317.300049    272.509757   
7    163.893924    86.354874   71.061623    512.146790      2.035320   
8    381.695689   315.468399    8.667033   1771.099976    111.775215   
9    947.580966  1067.646654   15.730861   3497.629883    136.620821   
10   811.442517   787.960874    8.746585   3201.749268      3.232403   
11   46

<ipython-input-5-0b0bf3e11a06>:142: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(pick_top_20)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>